### Введение в обработку естественного языка. Урок 5. Свёрточные сети для анализа текста.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from string import punctuation
import re
from tqdm.notebook import tqdm

In [3]:
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [4]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D
from tensorflow.keras.layers import Masking, SimpleRNN, LSTM, GRU
from tensorflow.keras.layers import GlobalMaxPool1D, AveragePooling1D, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.backend import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [5]:
# Загрузка данных
data = pd.read_excel("отзывы за лето.xls")
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20659 entries, 0 to 20658
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   20659 non-null  int64 
 1   Content  20656 non-null  object
 2   Date     20659 non-null  object
dtypes: int64(1), object(2)
memory usage: 484.3+ KB


In [7]:
def targ(x):
    r = 0
    if x > 3:
        r = 1
    return r

In [8]:
data['target'] = data['Rating'].apply(lambda x:  targ(x))

In [9]:
data.head(10)

,Rating,Content,Date,target
0,5,It just works!,2017-08-14,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,1
2,5,Отлично все,2017-08-14,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,1
4,5,"Очень удобно, работает быстро.",2017-08-14,1
5,5,Всё удобно норм 👍👍👍,2017-08-14,1
6,5,Очень удобное приложение.,2017-08-14,1
7,5,Все устраивает,2017-08-14,1
8,5,У меня работает все четко. В отличии от банком...,2017-08-14,1
9,5,Очень все хорошо👍,2017-08-14,1


In [10]:
target = data['target'].value_counts()
target

1    16724
0     3935
Name: target, dtype: int64

In [11]:
stop_words = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt).lower()
    txt = txt.replace("ё", "е")
    txt = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', txt)
    txt = re.sub('@[^\s]+', ' ', txt)
    txt = re.sub('[^a-zA-Zа-яА-Я]+', ' ', txt)
    txt = re.sub(' +', ' ', txt)
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in stop_words]
    return " ".join(txt)

data['Content'] = data['Content'].apply(preprocess_text)

In [12]:
data.head(10)

,Rating,Content,Date,target
0,5,it just works,2017-08-14,1
1,4,целое удобноной приложение минус хотеть большо...,2017-08-14,1
2,5,отлично,2017-08-14,1
3,5,зависать работа антивирус ранее пользоваться н...,2017-08-14,1
4,5,удобно работать быстро,2017-08-14,1
5,5,удобно норма,2017-08-14,1
6,5,удобный приложение,2017-08-14,1
7,5,устраивать,2017-08-14,1
8,5,работать чётко отличие банкомат вечно зависать...,2017-08-14,1
9,5,,2017-08-14,1


In [13]:
data = data[data['Content']!='']

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Content'], data['target'], test_size=0.2,
                                                    random_state=42, stratify=data['target'])

In [15]:
X_train

20133    вернуть старый версия программа сделать обязат...
9035     удобно быстро какой сложность использованиина ...
4625                                                 пипец
9264                                           превосходно
6312                                               отлично
                               ...                        
9781                 появиться антивирус приложение тупить
7666                                                удобно
11158                               отлично большой помощь
4392     тоня сохраняться сообщениян отправленныен вход...
20597                                         объновляться
Name: Content, Length: 15920, dtype: object

In [16]:
y_train

20133    0
9035     1
4625     0
9264     1
6312     1
        ..
9781     1
7666     1
11158    1
4392     1
20597    0
Name: target, Length: 15920, dtype: int64

In [17]:
text_corpus_train = X_train.values
text_corpus_test  = X_test.values

tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test  = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test  = pad_sequences(sequences_test,  maxlen=training_length)

y_train = y_train.values
y_test  = y_test.values

In [18]:
X_train

array([[   0,    0,    0, ...,   47,  347, 3810],
       [   0,    0,    0, ..., 1548, 2669,  158],
       [   0,    0,    0, ...,    0,    0, 1803],
       ...,
       [   0,    0,    0, ...,    5,   97,  419],
       [   0,    0,    0, ..., 9242,  799,   36],
       [   0,    0,    0, ...,    0,    0, 9243]])

### Model RNN

In [32]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Train on 14328 samples, validate on 1592 samples
Epoch 1/10
14328/14328 [==============================] - 3s 178us/sample - loss: 0.4805 - accuracy: 0.8065 - val_loss: 0.3806 - val_accuracy: 0.8241
Epoch 2/10
14328/14328 [==============================] - 2s 117us/sample - loss: 0.3565 - accuracy: 0.8490 - val_loss: 0.3149 - val_accuracy: 0.8631
Epoch 3/10
14328/14328 [==============================] - 2s 120us/sample - loss: 0.2895 - accuracy: 0.8850 - val_loss: 0.2611 - val_accuracy: 0.8744
Epoch 4/10
14328/14328 [==============================] - 2s 118us/sample - loss: 0.2226 - accuracy: 0.9125 - val_loss: 0.2345 - val_accuracy: 0.8945
Epoch 5/10
14328/14328 [==============================] - 2s 113us/sample - loss: 0.1793 - accuracy: 0.9308 - val_loss: 0.2484 - val_accuracy: 0.8920


In [34]:
score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

3981/3981 [==============================] - 0s 29us/sample - loss: 0.2430 - accuracy: 0.8980
Test score: 0.24295928266158628
Test accuracy: 0.89801556


### Model LSTM

In [35]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Train on 14328 samples, validate on 1592 samples
Epoch 1/10
14328/14328 [==============================] - 6s 436us/sample - loss: 0.5646 - accuracy: 0.7873 - val_loss: 0.4404 - val_accuracy: 0.8078
Epoch 2/10
14328/14328 [==============================] - 5s 332us/sample - loss: 0.3868 - accuracy: 0.8289 - val_loss: 0.3055 - val_accuracy: 0.8618
Epoch 3/10
14328/14328 [==============================] - 5s 336us/sample - loss: 0.2868 - accuracy: 0.8807 - val_loss: 0.2510 - val_accuracy: 0.8832
Epoch 4/10
14328/14328 [==============================] - 5s 330us/sample - loss: 0.2232 - accuracy: 0.9080 - val_loss: 0.2311 - val_accuracy: 0.8926
Epoch 5/10
14328/14328 [==============================] - 5s 334us/sample - loss: 0.1797 - accuracy: 0.9284 - val_loss: 0.2337 - val_accuracy: 0.8995


In [37]:
score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

3981/3981 [==============================] - 0s 88us/sample - loss: 0.2246 - accuracy: 0.9025
Test score: 0.22455416385461505
Test accuracy: 0.90253705


### Model GRU

In [53]:
model = Sequential()
model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [54]:
early_stopping=EarlyStopping(monitor='val_loss')

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Train on 14328 samples, validate on 1592 samples
Epoch 1/10
14328/14328 [==============================] - 7s 464us/sample - loss: 0.5617 - accuracy: 0.7843 - val_loss: 0.3994 - val_accuracy: 0.8078
Epoch 2/10
14328/14328 [==============================] - 5s 351us/sample - loss: 0.3517 - accuracy: 0.8238 - val_loss: 0.2767 - val_accuracy: 0.8668
Epoch 3/10
14328/14328 [==============================] - 5s 361us/sample - loss: 0.2570 - accuracy: 0.8944 - val_loss: 0.2484 - val_accuracy: 0.8857
Epoch 4/10
14328/14328 [==============================] - 5s 350us/sample - loss: 0.2072 - accuracy: 0.9190 - val_loss: 0.2411 - val_accuracy: 0.8938
Epoch 5/10
14328/14328 [==============================] - 5s 351us/sample - loss: 0.1670 - accuracy: 0.9380 - val_loss: 0.2424 - val_accuracy: 0.8926


In [55]:
score = model.evaluate(X_test, y_test, batch_size=512, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

3981/3981 [==============================] - 0s 69us/sample - loss: 0.2390 - accuracy: 0.8968
Test score: 0.23904869205925938
Test accuracy: 0.8967596


Модель LSTM показала наибольшую точность 90.25%